<a href="https://colab.research.google.com/github/Felhaba/Modelling/blob/main/CrossSectional_CreditSpreads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
path = '/content/drive/MyDrive/MathFinanceConsultingNatural/Modelling/'

### Importing data

In [ ]:
input_data = pd.read_excel(f'{path}InputParameters.xlsx')
cds_data = pd.read_excel(f'{path}CDS_Data.xlsx')

In [ ]:
#cds_data.head(5)
input_data.head(5)

,AAA,AA,A,BBB,BB,B,CCC,Asia,E.Eur,Europe,...,Consumer Services,Energy,Financials,Government,Healthcare,Industrials,Muni,Technology,Telecommunications Services,Utilities
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Cross-sectional method in a nutshell

&ensp; Proposed by Chourdakis et al. (2013) and is based on a cross-section methodology using a multidimensional linear regression. This approach still uses a categorisation based across rating, region, and sector (and potentially other categories), but it generates the spread via a factor approach rather than a direct mapping to the names in a given bucket. A given spread is generated as the product of several factors, such as:
* global ($M_{glob}$);
* industry sector ($M_{ind}$);
* region ($M_{region}$);
* rating ($M_{rating}$);  
* seniority ($M_{sen}$).

&ensp; Each factor will contain several sub-categories (e.g. different credit ratings), for which
the coefficients need to be estimated. The spread is defined by:
$$S^{generic}_i = M_{glob} \times M_{ind} \times M_{region} \times M_{rating} \times M_{sen}$$
or equivalently
$$\log(S^{generic}_i) = \log(M_{glob}) \times \log(M_{ind}) \times \log(M_{region}) \times \log(M_{rating}) \times \log(M_{sen})$$


&ensp; A key assumption behind such a methodology is that CDS spreads can be defined by a set of factors that are independent of the other components (e.g. there is a single multiplicative credit spread factor for all European credits, which is independent of the sector, rating, and seniority of those obligors).

### Modelling in python

#### 1. Adding a logarithm of CDS spread

In [ ]:
cds_data['LOG_CDS'] = np.log(cds_data['CDS'])

In [ ]:
cds_data['LOG_CDS']

0      -4.922624
1      -5.564890
2      -5.035953
3      -5.610292
4      -6.388961
          ...   
1067   -3.206453
1068   -2.253224
1069   -2.454571
1070   -3.999762
1071   -3.388479
Name: LOG_CDS, Length: 1072, dtype: float64

#### 2 Build a linear regression

In [ ]:
reg = LinearRegression().fit(input_data, cds_data['LOG_CDS'])

In [ ]:
reg.score(input_data, cds_data['LOG_CDS'])

0.6839725333479965

#### 3. Extract coefficients

In [ ]:
reg.coef_

array([-1.07712642e+13, -1.07712642e+13, -1.07712642e+13, -1.07712642e+13,
       -1.07712642e+13, -1.07712642e+13, -1.07712642e+13,  1.10284969e+13,
        1.10284969e+13,  1.10284969e+13,  1.10284969e+13,  1.10284969e+13,
        1.10284969e+13,  1.10284969e+13,  1.10284969e+13, -2.31582128e+12,
       -2.31582128e+12, -2.31582128e+12, -2.31582128e+12, -2.31582128e+12,
       -2.31582128e+12, -2.31582128e+12, -2.31582128e+12, -2.31582128e+12,
       -2.31582128e+12, -2.31582128e+12, -2.31582128e+12])

In [ ]:
intercept = reg.intercept_

In [ ]:
dict_coeffs = dict(map(lambda i,j : (i,j) , input_data.columns,reg.coef_))

#### Spread examples

In [ ]:
rating = input('Enter rating:')
region = input('Enter region:')
industry = input('Enter Industry:')
spread = round(100*np.exp(reg.intercept_ + dict_coeffs[rating] + dict_coeffs[region] + dict_coeffs[industry]), 3)

Enter rating:AAA
Enter region:N.Amer
Enter Industry:Financials


In [ ]:
print(f'Credit spread for {rating}, {region}, {industry} corporation is {spread}%')

Credit spread for AAA, N.Amer, Financials corporation is 0.238%
